In [1]:
import pandas as pd
import copy
import requests

In [2]:
# load in raw data
raw_data = pd.read_csv('./pre-study-responses.csv')

In [3]:
cols_to_keep = [
    'Name',
    'Email address',
    'Please select what things you would be interested in knowing about Coffee Shops.',
    'Please select what things you would be interested in knowing about Libraries (Mudd and Main Library). ',
    'Please select what things you would be interested in knowing about Gyms (SPAC and Bloomquist).',
    'Please select what things you would be interested in knowing about Free Food around campus.'
]

col_rename = ['name', 'email', 'coffeeshops', 'workspaces', 'gyms', 'freefood']

# clean data
user_preferences = raw_data[cols_to_keep]
user_preferences.columns = col_rename
user_preferences.loc[:, 'first_name'] = user_preferences['name'].apply(lambda x: x.split(' ')[0])
user_preferences.loc[:, 'last_name'] = user_preferences['name'].apply(lambda x: x.split(' ')[1])
user_preferences = user_preferences.loc[:, ['name', 'first_name', 'last_name', 'email',
                                     'coffeeshops', 'workspaces', 'gyms', 'freefood']]
user_preferences.head(10)

/Users/kapilgarg/miniconda3/envs/les/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kapilgarg/miniconda3/envs/les/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,name,first_name,last_name,email,coffeeshops,workspaces,gyms,freefood
0,Stephanie Ger,Stephanie,Ger,stephanieger@u.northwestern.edu,"Availability of private work TABLES, Seating n...",I do not want to know any information about Wo...,Availability of squat racks to use with free w...,I want to know there is free food IF the food ...
1,Hyunjee Lee,Hyunjee,Lee,hyunjeelee2018@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Seating n...",Availability of squat racks to use with free w...,I want to know there is free food IF the food ...
2,Vanessa Tang,Vanessa,Tang,vanessatang2019@u.northwestern.edu,Availability of private work TABLES,Availability of private work TABLES,Availability of free weights (dumbbells or bar...,I want to know there is free food IF the food ...
3,Josh Shi,Josh,Shi,joshuashi2018@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...",I do not want to know any information about wo...,I want to know there is free food IF the food ...
4,Jennie Werner,Jennie,Werner,jw18@u.northwestern.edu,I do not want to know any information about Co...,I do not want to know any information about Wo...,I do not want to know any information about wo...,I want to know there is free food IF the food ...
5,Maggie Lou,Maggie,Lou,margaretlou2019@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...","Availability of treadmills, Availability of st...",I want to know there is free food IF the food ...
6,Meredith Sonnefeldt,Meredith,Sonnefeldt,meredithsonnefeldt2019@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...","Availability of treadmills, Availability of el...",I want to know there is free food IF the food ...
7,Raven Watson,Raven,Watson,ravenwatson2019@u.northwestern.edu,Availability of private work TABLES,I do not want to know any information about Wo...,I do not want to know any information about wo...,I want to know there is free food IF the food ...
8,Hamilton Vuu,Hamilton,Vuu,hamiltonvuu2021@u.northwestern.edu,I do not want to know any information about Co...,"Availability of private work TABLES, Availabil...",Availability of benches for use with free weig...,I want to know there is free food IF the food ...
9,meg grasse,meg,grasse,meggyg@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...","Availability of treadmills, Availability of fr...",I want to know there is free food IF the food ...


In [4]:
# output scaffold
preference_template = {
   "coffeeshop":{
      "privateseating":[],
      "privateseatingoutlets":[],
      "privateseatingwindows":[],
      "sharedseating":[],
      "sharedseatingoutlets":[],
      "sharedseatingwindows":[]
   },
   "gym":{
      "treadmills":[],
      "ellipticals":[],
      "freeweights":[],
      "freeweightsbenches":[],
      "freeweightssquatracks":[],
      "stretch":[],
      "stretchmats":[],
      "stretchrollers":[]
   },
   "workspace":{
      "privateseating":[],
      "privateseatingoutlets":[],
      "sharedseating":[],
      "sharedseatingoutlets":[],
      "whiteboards":[],
      "whiteboardsmarkers":[]
   },
   "freefood":{
      "foodtype":[],
      "stillleft":[]
   }
}

In [5]:
def map_preferences_for_coffeeshop(response):
    preference_list = response.split(', ')
    preference_object = copy.deepcopy(preference_template['coffeeshop'])
    
    # deal swiftly with "I do not..."
    if 'I do not want to know any information about Coffee Shops' in preference_list:
        return preference_object
    
    # iterate through
    for preference_item in preference_list:
        if preference_item == 'Availability of private work TABLES':
            preference_object['privateseating'].append('tables')
        elif preference_item == 'Availability of private work COUCHES/CHAIRS':
            preference_object['privateseating'].append('couches/chairs')
        elif preference_item == 'Availability of private work TABLES or COUCHES/CHAIRS':
            preference_object['privateseating'].append('tables and couches/chairs')
            if 'tables' not in preference_object['privateseating']:
                preference_object['privateseating'].append('tables')
            if 'couches/chairs' not in preference_object['privateseating']:
                preference_object['privateseating'].append('couches/chairs')
                
        elif preference_item == 'Availability of communal or shared work tables':
            preference_object['sharedseating'].append('yes')
            
        elif preference_item == 'Seating near outlets (must also select a kind of seating above)':
            if len(preference_object['privateseating']) != 0:
                preference_object['privateseatingoutlets'].append('yes')
            if len(preference_object['sharedseating']) != 0:
                preference_object['sharedseatingoutlets'].append('yes')
        elif preference_item == 'Seating near windows  (must also select a kind of seating above)':
            if len(preference_object['privateseating']) != 0:
                preference_object['privateseatingwindows'].append('yes')
            if len(preference_object['sharedseating']) != 0:
                preference_object['sharedseatingwindows'].append('yes')
                
    return preference_object

In [6]:
def map_preferences_for_workspace(response):
    preference_list = response.split(', ')
    preference_object = copy.deepcopy(preference_template['workspace'])
    
    # deal swiftly with "I do not..."
    if 'I do not want to know any information about Workspaces' in preference_list:
        return preference_object
    
    # iterate through
    for preference_item in preference_list:
        if preference_item == 'Availability of private work TABLES':
            preference_object['privateseating'].append('tables')
        elif preference_item == 'Availability of private work COUCHES/CHAIRS':
            preference_object['privateseating'].append('couches/chairs')
        elif preference_item == 'Availability of private work TABLES or COUCHES/CHAIRS':
            preference_object['privateseating'].append('tables and couches/chairs')
            if 'tables' not in preference_object['privateseating']:
                preference_object['privateseating'].append('tables')
            if 'couches/chairs' not in preference_object['privateseating']:
                preference_object['privateseating'].append('couches/chairs')
                
        elif preference_item == 'Availability of communal or shared work tables':
            preference_object['sharedseating'].append('yes')
            
        elif preference_item == 'Seating near outlets (must also select a kind of seating above)':
            if len(preference_object['privateseating']) != 0:
                preference_object['privateseatingoutlets'].append('yes')
            if len(preference_object['sharedseating']) != 0:
                preference_object['sharedseatingoutlets'].append('yes')
        elif preference_item == 'Seating near windows  (must also select a kind of seating above)':
            if len(preference_object['privateseating']) != 0:
                preference_object['privateseatingwindows'].append('yes')
            if len(preference_object['sharedseating']) != 0:
                preference_object['sharedseatingwindows'].append('yes')
                
        elif preference_item == 'Availability of whiteboards':
            preference_object['whiteboards'].append('yes')
        elif preference_item == 'Availability of dry erase markers near whiteboards':
            preference_object['whiteboardsmarkers'].append('yes')
            if len(preference_object['whiteboards']) == 0:
                preference_object['whiteboards'].append('yes')
                
    return preference_object

In [7]:
def map_preferences_for_gym(response):
    preference_list = response.split(', ')
    preference_object = copy.deepcopy(preference_template['gym'])
    
    # deal swiftly with "I do not..."
    if 'I do not want to know any information about workout equipment at Gyms' in preference_list:
        return preference_object
    
    # iterate through
    for preference_item in preference_list:
        if preference_item == 'Availability of treadmills':
            preference_object['treadmills'].append('yes')
        elif preference_item == 'Availability of ellipticals':
            preference_object['ellipticals'].append('yes')
            
        elif preference_item == 'Availability of free weights (dumbbells or barbells)':
            preference_object['freeweights'].append('yes')
        elif preference_item == 'Availability of benches for use with free weights':
            preference_object['freeweightsbenches'].append('yes')
            if len(preference_object['freeweights']) == 0:
                preference_object['freeweights'].append('yes')
        elif preference_item == 'Availability of squat racks to use with free weights':
            preference_object['freeweightssquatracks'].append('yes')
            if len(preference_object['freeweights']) == 0:
                preference_object['freeweights'].append('yes')
                
        elif preference_item == 'Availability of stretching space':
            preference_object['stretch'].append('yes')
        elif preference_item == 'Availability of mats to stretch on':
            preference_object['stretchmats'].append('yes')
            if len(preference_object['stretch']) == 0:
                preference_object['stretch'].append('yes')
        elif preference_item == 'Availability of rollers to stretch with':
            preference_object['stretchrollers'].append('yes')
            if len(preference_object['stretch']) == 0:
                preference_object['stretch'].append('yes')
                
    return preference_object

In [8]:
def map_preferences_for_freefood(response):
    preference_list = response.split(', ')
    preference_object = copy.deepcopy(preference_template['freefood'])
    
    # deal swiftly with "I do not..."
    if 'I do not want to know any information about Free Food around campus' in preference_list:
        return preference_object
    
    # iterate through
    for preference_item in preference_list:
        if preference_item == 'I want to know there is free food IF the food is Pizzas':
            preference_object['foodtype'].append('pizza')
        elif preference_item == 'I want to know there is free food IF the food is Bagels':
            preference_object['foodtype'].append('bagels')
        elif preference_item == 'I want to know there is free food IF the food is Donuts':
            preference_object['foodtype'].append('donuts')
                
    return preference_object

In [ ]:
for (index, row) in user_preferences.iterrows():
    # create data to upload
    upload_data = {
        'username': row['email'],
        'email': row['email'],
        'password': 'delta',
        'firstName': row['first_name'],
        'lastName': row['last_name'],
        'vendorId': '',
        'pushToken': '',
        'lastLoggedIn': '',
        'lastLoggedOut': '',
        'preferences': {
            'coffeeshop': map_preferences_for_coffeeshop(row['coffeeshops']),
            'gym': map_preferences_for_gym(row['gyms']),
            'workspace': map_preferences_for_workspace(row['workspaces']),
            'freefood': map_preferences_for_freefood(row['freefood'])
        }
    }
    
    r = requests.post('http://localhost:5000/parse/users',
                      headers={
                          'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'
                      },
                      json=upload_data)
    print(r.status_code, r.reason, '\n')

In [150]:
# add an account for kapil
upload_data = {
    'username': 'kg@les.com',
    'email': 'kg@les.com',
    'password': 'delta',
    'firstName': 'Kapil',
    'lastName': 'Garg',
    'vendorId': '',
    'pushToken': '',
    'lastLoggedIn': '',
    'lastLoggedOut': '',
    'preferences': {
       'coffeeshop':{
          'privateseating':[
             'tables'
          ],
          'privateseatingoutlets':[
             'no'
          ],
          'privateseatingwindows':[
              'yes'
          ],
          'sharedseating':[
             'yes'
          ],
          'sharedseatingoutlets':[
             'no'
          ],
          'sharedseatingwindows':[
             'yes'
          ]
       },
       'gym':{
          'treadmills':[
             'yes'
          ],
          'ellipticals':[
             'yes'
          ],
          'freeweights':[
             'yes'
          ],
          'freeweightsbenches':[
             'yes'
          ],
          'freeweightssquatracks':[
             'yes'
          ],
          'stretch':[
             'yes'
          ],
          'stretchmats':[
             'yes'
          ],
          'stretchrollers':[
             'yes'
          ]
       },
       'workspace':{
          'privateseating':[
             'tables',
             'couches/chairs',
             'tables and couches/chairs'
          ],
          'privateseatingoutlets':[
             'yes'
          ],
          'sharedseating':[
              'yes'
          ],
          'sharedseatingoutlets':[
              'yes'
          ],
          'whiteboards':[
             'yes'
          ],
          'whiteboardsmarkers':[
             'yes'
          ]
       },
       'freefood':{
          'foodtype':[
            'pizza',
            'donuts',
            'bagels'
          ],
          'stillleft':[

          ]
       }
    }
}

r = requests.post('http://localhost:5000/parse/users',
                  headers={
                      'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'
                  },
                  json=upload_data)
print(r.status_code, r.reason, '\n')

201 Created 



In [16]:
for (index, row) in user_preferences[-2:].iterrows():
    # create data to upload
    upload_data = {
        'username': row['email'],
        'email': row['email'],
        'password': 'delta',
        'firstName': row['first_name'],
        'lastName': row['last_name'],
        'vendorId': '',
        'pushToken': '',
        'lastLoggedIn': '',
        'lastLoggedOut': '',
        'preferences': {
            'coffeeshop': map_preferences_for_coffeeshop(row['coffeeshops']),
            'gym': map_preferences_for_gym(row['gyms']),
            'workspace': map_preferences_for_workspace(row['workspaces']),
            'freefood': map_preferences_for_freefood(row['freefood'])
        }
    }
    
    for url in ['http://localhost:5000', 'https://les-4x.herokuapp.com',
                'https://les-opplocation.herokuapp.com', 'https://les-oppdistance.herokuapp.com']:    
        r = requests.post(url + '/parse/users',
                          headers={
                              'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'
                          },
                          json=upload_data)
        print(r.status_code, r.reason, '\n')

201 Created 

201 Created 

201 Created 

201 Created 

201 Created 

201 Created 

201 Created 

201 Created 



In [14]:
user_preferences[-2:]

,name,first_name,last_name,email,coffeeshops,workspaces,gyms,freefood
120,Kathryn Donermeyer,Kathryn,Donermeyer,kathryndonermeyer2021@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...","Availability of treadmills, Availability of el...",I want to know there is free food IF the food ...
121,Ally O'Donnell,Ally,O'Donnell,allisonodonnell2019@u.northwestern.edu,"Availability of private work TABLES, Availabil...","Availability of private work TABLES, Availabil...",Availability of squat racks to use with free w...,I want to know there is free food IF the food ...
